# Planet Physics Simulation
#### <i class="fa fa-exclamation-circle"></i> Disclaimer: Do not modify this iPython notebook.  If you want to modify it, copy it to a different directory, one that is not automatically updated, and modify it there

The goal of this iPython notebook is to create a planet simulation demo 

First we need to import some python modules

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

We define the planet object, in order to assign atributes'with the __init__ function


In [ ]:
class planet:
    def __init__(self,m,v,p,F,a):
        self.mass = m
        self.vel = v
        self.position = p
        self.acceleration = a
        self.force = F


We next define several functions, which update the acceleration, velocities, and positions of the planetary system

In [ ]:
#This function updates the acceleration of the planets. 
def update_acceleration(planet_list):
    total, = np.shape(planet_list) #total number of planets
    for i in range(total-1): #for each pairwise interaction between planets
        for j in range(i+1,total):
            #Since the first element in the list is a fixed planet (e.g. earth), we don't update the force on it.
            if i == 0 :
                rij = np.subtract(planet_list[i].position, planet_list[j].position) #get a position vector
                distance = np.sqrt(np.dot(rij,rij)) #distance between earth and planet j
                magnitude_f = (0.2458*planet_list[i].mass * planet_list[j].mass/np.power(distance,2)) # get magnitude of force
                vect_force = np.multiply(magnitude_f,rij) #Force vector on planet j 
                planet_list[j].force = vect_force #
           #calculate the forces between all pairs of planets
            else :
                rij = np.subtract(planet_list[i].position, planet_list[j].position) #get a position vector
                distance = np.sqrt(np.dot(rij,rij)) #distance between two planets and planet j
                magnitude_f = (0.2458*planet_list[i].mass * planet_list[j].mass/np.power(distance,2)) # get magnitude of force
                vect_force = np.multiply(magnitude_f,rij) #Force vector on planet j 
                planet_list[j].force = vect_force + planet_list[j].force # update forces on planet j
                planet_list[i].force = np.multiply(-1,vect_force) + planet_list[i].force # update forces on planet i
            
    for i in range(1,total):
        planet_list[i].acceleration = np.multiply(1/planet_list[i].mass, planet_list[i].force) # a = F/m

In [ ]:
#Next, the function that updates the velocities:
def update_vel(planet_list,t_step):
    total, = np.shape(planet_list) # total number of planets
    for i in range(1,total): #for each planet
        planet_list[i].vel = planet_list[i].vel + t_step * planet_list[i].acceleration #simple finite differences update

In [ ]:
#And the function that updates the positions:
def update_pos(planet_list,t_step):
    total, = np.shape(planet_list) # total number of planets
    for i in range(1,total): # for each planet
        planet_list[i].position = planet_list[i].position + t_step * planet_list[i].vel # simple finite differences update
        


Next, we define our parameters: 

In [ ]:
#time
time=300

######Earth######
mass_earth = 81.3
vel_earth = 0
pos_earth = [0,0]

#####MOON#####
mass_moon = 1
vel_moon = [0,0.578] #velocity vector [vx, vy]
pos_moon = [40,0]   #position vector

####ASTEROID#####
mass_asteroid = 1  #0.1
vel_asteroid = [0.5,0]   #velocity vector
pos_asteroid = [0,52]     #position vector

Let's initialized the planet objects by calling the class we created way at the top!

In [ ]:
earth = planet(mass_earth,vel_earth,pos_earth,[0,0],[0,0]) 
moon = planet(mass_moon,vel_moon,pos_moon,[0,0],[0,0])
asteroid = planet(mass_asteroid,vel_asteroid,pos_asteroid,[0,0],[0,0])

system = []
system.append(earth)
system.append(moon)
system.append(asteroid)

Run the simulation for our toy planetary system

In [ ]:
t_step = 0.1 # define the time step
niter = int(time/t_step) # The number of iterations is the total time divided by the time step
data_moon=np.zeros([niter,2]) # a matrix of zeros, where we'll hold all the positions for the moon
data_asteroid = np.zeros([niter,2]) # a matrix of zeros, where we'll hold all positions for the asteroid

for i in range(niter): # for each iteration
    update_acceleration(system) # update the system's acceleration
    update_vel(system,t_step) # update the velocities
    update_pos(system,t_step) # update the positions
    data_moon[i,:] = system[1].position #store the moon's positions in the matrix
    data_asteroid[i,:] = system[2].position # store the asteroid's position in the matrix
    

And below, we create an animation of our planetary simulation

In [ ]:
rate = 10
# Define a circle object
circle1=plt.Circle((0,0),10,color='b')
#figure where animation will take place
fig = plt.figure()
#Axes of figure
ax = plt.axes(xlim=(-100, 100), ylim=(-100, 100))
#Add a circle to figure
ax.add_patch(circle1)
#Aspect ration of x and y axes
ax.set_aspect('equal')
#Define some properties of the objects to animate
line1, = ax.plot([], [], 'go',lw=1)
line2, = ax.plot([],[],'ro',lw=1)

#Initialize each frame as empty
def init():
    line1.set_data([], [])
    line2.set_data([],[])
    return line1, line2

#animate function, which returs the animated object in each frame
def animate(i):
    line1.set_data(data_moon[i*rate,0],data_moon[i*rate,1] ) #
    line2.set_data(data_asteroid[i*rate,0],data_asteroid[i*rate,1] ) # 
    return line1, line2

#call the animation function
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=niter/rate, interval=1,blit=False)

Now let's create a function to animate the planets as an .mp4 movie... there is some magic behind the scenes here, Don't worry about it too much.

**If curious:** First we create a temporary file to hold our movie, then we draw each frame of the movie and then we embed this in HTML code bacause Jupyter is a web-based platform.



In [ ]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

 Now we can run our function, like your movie? Right-click it to save.

In [ ]:
# call our new function to display the animation
display_animation(anim)